# PockDock

This notebook demonstrates how a variety of different tools can be glued together into an efficient and flexible workflow using **Crossflow**.

The workflow downloads a protein-ligand complex form the PDB, runs fpocket, then docks the ligand back into the biggest pocket found. Then it calculates the error between the crystal structure coordinates of the ligand and those of each docking pose, before and after least-squares fitting.

The notebook requires you to have versions of **fpocket**, **autodock tools** and **autodock vina** installed on the worker node(s) of your dask cluster, which is up and running and identifiable via the file "dask.dat" in the current directory (created when dask-scheduler is started with the `--scheduler_file` option)

In [1]:
from crossflow import filehandling, kernels, clients
import sys
from urllib.request import urlretrieve
import numpy as np
import mdtraj as mdt

Create a crossflow client, connected to a local pool of workers:

In [2]:
client = clients.Client(scheduler_file='dask.dat')
client.client

Client Scheduler: tcp://192.168.1.67:8786 Dashboard: http://192.168.1.67:8787/status,Cluster Workers: 1 Cores: 8 Memory: 17.18 GB


Make the SubprocessKernels for **fpocket** and **Vina**, and FunctionKernels for other tasks:

In [3]:
# The fpocket kernel:
fpocket = kernels.SubprocessKernel('fpocket -f x.pdb')
fpocket.set_inputs(['x.pdb'])
fpocket.set_outputs(['x_out/x_out.pdb'])

In [4]:
# The vina kernel:
vina = kernels.SubprocessKernel('vina --receptor r.pdbqt --ligand l.pdbqt --out out.pdbqt --log dock.log'
                                 ' --center_x {xc} --center_y {yc} --center_z {zc}'
                                 ' --size_x {sx} --size_y {sy} --size_z {sz}')
vina.set_inputs(['r.pdbqt', 'l.pdbqt', 'xc', 'yc', 'zc', 'sx', 'sy', 'sz'])
vina.set_outputs(['out.pdbqt', 'dock.log'])

In [5]:
# AutoDock Tool based kernels to prepare receptor and ligand for docking:
prep_receptor = kernels.SubprocessKernel('adt prepare_receptor4.py -r x.pdb -o x.pdbqt')
prep_receptor.set_inputs(['x.pdb'])
prep_receptor.set_outputs(['x.pdbqt'])

prep_ligand = kernels.SubprocessKernel('adt prepare_ligand4.py -l x.pdb -o x.pdbqt')
prep_ligand.set_inputs(['x.pdb'])
prep_ligand.set_outputs(['x.pdbqt'])

In [79]:

def _download_and_split(pdb_code, ligand_residue_name):
    '''
    A function to download a pdb file, and split into receptor and ligand
    
    Args:
        pdb_code (str): 4-letter PDB code
        ligand_residue_name (str): 3-letter residue name
        
    Returns:
        mdt.trajectory: the receptor (protein atoms only)
        mdt.trajectory: the ligand
    '''
    pdb_file = pdb_code + '.pdb'
    path = urlretrieve('http://files.rcsb.org/download/' + pdb_file, pdb_file)
    hydrated_complex = mdt.load(pdb_file)
    receptor_atoms = hydrated_complex.topology.select('protein and chainid 0')
    found = False
    for chain in hydrated_complex.topology.chains:
        for r in chain.residues:
            if r.name == ligand_residue_name and not found:
                cid = chain.index
                found = True

    ligand_atoms = hydrated_complex.topology.select('resname MAU and chainid {}'.format(cid))
    receptor = mdt.load(pdb_file, atom_indices=receptor_atoms)
    ligand = mdt.load(pdb_file, atom_indices=ligand_atoms)
    return receptor, ligand
# Now make a FunctionKernel for it:
download_and_split = kernels.FunctionKernel(_download_and_split)
download_and_split.set_inputs(['pdb_code', 'ligand_residue_name'])
download_and_split.set_outputs(['receptor', 'ligand'])

In [ ]:

def _pdbqt2pdb(infile):
    '''
    A Function to convert pdbqt files back to pdb ones
    
    Args:
        infile (str): name of the input file, .pdbqt format
    
    Returns:
        str: name of the .pdb file (always 'tmp.pdb')
    '''
    outfile = 'tmp.pdb'
    fout = open(outfile, 'w')
    with open(infile, 'r') as fin:
        for line in fin:
            if line[1:6] in 'ATOM  MODEL ENDMDL':
                fout.write(line)       
    fout.close()
    return 'tmp.pdb'

# Now make a FunctionKernel for this:
pdbqt2pdb = kernels.FunctionKernel(_pdbqt2pdb)
pdbqt2pdb.set_inputs(['infile'])
pdbqt2pdb.set_outputs(['outfile'])

In [8]:
def _get_dimensions(pockets):
    '''
    A Function to find the centre and extents of the largest pocket found by fpocket
    
    Args:
        pockets (str): Name of the pdb format file produced by fpocket
        
    Returns:
        (float,) * 6: the pocket centre and extents in x/y/z - in Angstroms
    '''
    buffer = 2.0
    t = mdt.load(pockets)
    site = t.topology.select('resname STP and residue 1') # This should be the largest pocket
    # In the next two lines, the factor of 10 is a conversion from nanometres to Angstroms:
    xc, yc, zc = tuple(10 * (t.xyz[0][site].min(axis=0) + t.xyz[0][site].max(axis=0)) / 2)
    sx, sy, sz = tuple(10 * (t.xyz[0][site].max(axis=0) - t.xyz[0][site].min(axis=0)) + buffer)
    return xc, yc, zc, sx, sy, sz

# Now make a FunctionKernel for this:
get_dimensions = kernels.FunctionKernel(_get_dimensions)
get_dimensions.set_inputs(['pockets'])
get_dimensions.set_outputs(['xc', 'yc', 'zc', 'sx', 'sy', 'sz'])

Now we construct the workflow. For convenience it's split up here into sections.

In [80]:
#pdb_code = '1qy1'
#ligand_residue_name = 'PRZ'
pdb_code = '1ha3'
ligand_residue_name = 'MAU'

In [81]:
receptor, ligand = client.submit(download_and_split, pdb_code, ligand_residue_name)
print(ligand.result())

<mdtraj.Trajectory with 1 frames, 58 atoms, 1 residues, and unitcells>


In [82]:
# Run fpocket:
pockets = client.submit(fpocket, receptor)

In [83]:
# Find the dimensions of the biggest pocket
xc, yc, zc, sx, sy, sz = client.submit(get_dims, pockets)

In [84]:
# Prepare receptor and ligand for docking:
receptor_qt = client.submit(prep_receptor, receptor)
ligand_qt = client.submit(prep_ligand, ligand)

In [85]:
# Run vina:
docks, logfile = client.submit(vina, receptor_qt, ligand_qt, xc, yc, zc, sx, sy, sz)

In [86]:
# Check the log file:
print(logfile.result().read_text())

#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 4 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: -16

In [87]:
# Convert the docked poses back to PDB format, and calculate unfitted and fitted rmsds using MDTraj:
pdbout = client.submit(pdbqt_to_pdb, docks)
docktraj = mdt.load(str(pdbout.result()))
dxyz = docktraj.xyz - ligand.result().xyz
dxyz = (dxyz * dxyz).sum(axis=2).mean(axis=1)
rmsd = mdt.rmsd(docktraj, ligand.result()) * 10.0
unfitted_rmsd = np.sqrt(dxyz) * 10.0
print('Mode Fitted   Unfitted')
print('      rmsd      rmsd')
for mode in range(len(docktraj)):
    print('{:3d}   {:5.3f}    {:6.3f}'.format(mode+1, rmsd[mode], unfitted_rmsd[mode]))

Mode Fitted   Unfitted
      rmsd      rmsd
  1   6.887    27.437
  2   7.676    27.961
  3   7.216    27.924
  4   7.710    28.600
  5   6.692    30.023
  6   6.843    28.025
  7   7.461    28.657
  8   7.682    28.948
  9   6.881    28.493


Now let's run at scale. For this we will have a new output format that puts the results for each protein on a single line:

In [89]:
def report(pdb_code, ligand_residue_name, pdbout, reference):
    docktraj = mdt.load(str(pdbout.result()))
    dxyz = docktraj.xyz - reference.result().xyz
    dxyz = (dxyz * dxyz).sum(axis=2).mean(axis=1)
    unfitted_rmsd = np.sqrt(dxyz) * 10.0
    fmt = '{} {} ' + '{:7.3f}' * len(docktraj)
    print(fmt.format(pdb_code, ligand_residue_name, *unfitted_rmsd))
report(pdb_code, ligand_residue_name, pdbout, ligand)

1ha3 MAU  27.437 27.961 27.924 28.600 30.023 28.025 28.657 28.948 28.493


In [48]:
pdb_code = '1ha3'
pdb_file = pdb_code + '.pdb'
path = urlretrieve('http://files.rcsb.org/download/' + pdb_file, pdb_file)

In [49]:
hydrated_complex = mdt.load(pdb_file)
receptor_atoms = hydrated_complex.topology.select('protein and chainid 0')
ligand_atoms = hydrated_complex.topology.select('resname {} and chainid 0'.format(ligand_residue_name))
print(receptor_atoms)
print(ligand_atoms)

[   0    1    2 ... 3033 3034 3035]
[]


In [68]:
for chain in hydrated_complex.topology.chains:
    for r in chain.residues:
        if r.name == 'MAU':
            cid = chain.index

ligand_atoms = hydrated_complex.topology.select('resname MAU and chainid {}'.format(cid))
print(ligand_atoms)
        

[6096 6097 6098 6099 6100 6101 6102 6103 6104 6105 6106 6107 6108 6109
 6110 6111 6112 6113 6114 6115 6116 6117 6118 6119 6120 6121 6122 6123
 6124 6125 6126 6127 6128 6129 6130 6131 6132 6133 6134 6135 6136 6137
 6138 6139 6140 6141 6142 6143 6144 6145 6146 6147 6148 6149 6150 6151
 6152 6153]
